In [1]:
import pandas as p
import numpy as n
import matplotlib.pyplot as plt

In [2]:
data=p.read_csv('NY property data.csv')
data.head()

,RECORD,BBLE,BORO,BLOCK,LOT,EASEMENT,OWNER,BLDGCL,TAXCLASS,LTFRONT,...,BLDFRONT,BLDDEPTH,AVLAND2,AVTOT2,EXLAND2,EXTOT2,EXCD2,PERIOD,YEAR,VALTYPE
0,1,1000010101,1,1,101,NaN,U S GOVT LAND & BLDGS,P7,4,500,...,0,0,3775500.0,8613000.0,3775500.0,8613000.0,NaN,FINAL,2010/11,AC-TR
1,2,1000010201,1,1,201,NaN,U S GOVT LAND & BLDGS,Z9,4,27,...,0,0,11111400.0,80690400.0,11111400.0,80690400.0,NaN,FINAL,2010/11,AC-TR
2,3,1000020001,1,2,1,NaN,DEPT OF GENERAL SERVI,Y7,4,709,...,709,564,32321790.0,40179510.0,32321790.0,40179510.0,NaN,FINAL,2010/11,AC-TR
3,4,1000020023,1,2,23,NaN,DEPARTMENT OF BUSINES,T2,4,793,...,85,551,13644000.0,15750000.0,13644000.0,15750000.0,NaN,FINAL,2010/11,AC-TR
4,5,1000030001,1,3,1,NaN,PARKS AND RECREATION,Q1,4,323,...,89,57,106348680.0,107758350.0,106348680.0,107758350.0,NaN,FINAL,2010/11,AC-TR


In [3]:
r=data['OWNER'].value_counts().head(20).index.tolist()
r

['PARKCHESTER PRESERVAT',
 'PARKS AND RECREATION',
 'DCAS',
 'HOUSING PRESERVATION',
 'CITY OF NEW YORK',
 'DEPT OF ENVIRONMENTAL',
 'BOARD OF EDUCATION',
 'NEW YORK CITY HOUSING',
 'CNY/NYCTA',
 'NYC HOUSING PARTNERSH',
 'YORKVILLE TOWERS ASSO',
 'DEPARTMENT OF BUSINES',
 'DEPT OF TRANSPORTATIO',
 'MTA/LIRR',
 'PARCKHESTER PRESERVAT',
 'MH RESIDENTIAL 1, LLC',
 '434 M LLC',
 'LINCOLN PLAZA ASSOCIA',
 'DEUTSCHE BANK NATIONA',
 '561 11TH AVENUE TMG L']

In [4]:
r.remove('YORKVILLE TOWERS ASSO')
r.remove('434 M LLC')
r.remove('DEUTSCHE BANK NATIONA')
r.remove('561 11TH AVENUE TMG L')
r.append('U S GOVERNMENT OWNRD')
r.append('UNITED STATES OF AMER')
r.append('THE CITY OF NEW YORK')
r.append('NYS URBAN DEVELOPMENT')
r.append('NYS DEPT OF ENVIRONME')
r.append('CULTURAL AFFAIRS')
r.append('DEPT OF GENERAL SERVI')
r.append('DEPT RE_CITY OF NY')

In [5]:
r

['PARKCHESTER PRESERVAT',
 'PARKS AND RECREATION',
 'DCAS',
 'HOUSING PRESERVATION',
 'CITY OF NEW YORK',
 'DEPT OF ENVIRONMENTAL',
 'BOARD OF EDUCATION',
 'NEW YORK CITY HOUSING',
 'CNY/NYCTA',
 'NYC HOUSING PARTNERSH',
 'DEPARTMENT OF BUSINES',
 'DEPT OF TRANSPORTATIO',
 'MTA/LIRR',
 'PARCKHESTER PRESERVAT',
 'MH RESIDENTIAL 1, LLC',
 'LINCOLN PLAZA ASSOCIA',
 'U S GOVERNMENT OWNRD',
 'UNITED STATES OF AMER',
 'THE CITY OF NEW YORK',
 'NYS URBAN DEVELOPMENT',
 'NYS DEPT OF ENVIRONME',
 'CULTURAL AFFAIRS',
 'DEPT OF GENERAL SERVI',
 'DEPT RE_CITY OF NY']

In [6]:
r.remove('DEPT RE_CITY OF NY')
r.append('DEPT RE-CITY OF NY')

In [7]:
sample=data[~data['OWNER'].isin(r)].reset_index(drop=True)

In [8]:
sample.shape

(1046826, 32)

In [9]:
ans=sample.copy()

### Missing value imputation

#### ZIP

In [10]:
m=n.where(p.isnull(sample['ZIP']))[0]
len(m)

21772

In [11]:
for i in range(len(m)):
    sample.loc[m[i],'ZIP']=sample.loc[m[i]-1,'ZIP']
        

In [12]:
m=n.where(p.isnull(sample['ZIP']))[0]
len(m)

0

#### STORIES

In [13]:
t=sample.groupby('TAXCLASS')['STORIES'].mean()
t

TAXCLASS
1      2.111633
1A     1.656867
1B     4.000000
1C     3.052748
1D     1.068966
2     16.310549
2A     2.844032
2B     4.001635
2C     4.744977
3      1.000000
4      5.517257
Name: STORIES, dtype: float64

In [14]:
sample['STORIES']=sample['STORIES'].fillna(value=0)

In [15]:
(sample['STORIES']==0).sum()

43968

In [16]:
for i in t.index:
    sample.loc[(sample['STORIES']==0)&(sample['TAXCLASS']==i),'STORIES']=t[i]

In [17]:
sample['STORIES']

0           5.517257
1           5.517257
2          50.000000
3          50.000000
4          50.000000
             ...    
1046821     2.500000
1046822     2.500000
1046823     2.000000
1046824     2.000000
1046825     2.000000
Name: STORIES, Length: 1046826, dtype: float64

In [18]:
(sample['STORIES']==0).sum()

0

#### FULLVAL, AVLAND, AVTOT


In [19]:
sample['FULLVAL'].isna().sum()

0

In [20]:
temp=sample[sample['FULLVAL']!=0]
f=temp.groupby('TAXCLASS')['FULLVAL'].mean()

In [21]:
temp=sample[sample['AVLAND']!=0]
a=temp.groupby('TAXCLASS')['AVLAND'].mean()

In [22]:
temp=sample[sample['AVTOT']!=0]
avt=temp.groupby('TAXCLASS')['AVTOT'].mean()

In [23]:
sample['FULLVAL']=sample['FULLVAL'].fillna(value=0)
sample['AVLAND']=sample['AVLAND'].fillna(value=0)
sample['AVTOT']=sample['AVTOT'].fillna(value=0)

In [24]:
for i in f.index:
    sample.loc[(sample['FULLVAL']==0)&(sample['TAXCLASS']==i),'FULLVAL']=f[i]
    sample.loc[(sample['AVLAND']==0)&(sample['TAXCLASS']==i),'AVLAND']=a[i]
    sample.loc[(sample['AVTOT']==0)&(sample['TAXCLASS']==i),'AVTOT']=avt[i]

#### LOT AND BUILDING SIZE

In [25]:
z=['LTFRONT','LTDEPTH','BLDFRONT','BLDDEPTH']
for i in z:
    sample.loc[sample[i]==0,i]=n.nan
    sample.loc[sample[i]==1,i]=n.nan
    

In [26]:
ltf=sample.groupby('TAXCLASS')['LTFRONT'].mean()
ltd=sample.groupby('TAXCLASS')['LTDEPTH'].mean()
bldf=sample.groupby('TAXCLASS')['BLDFRONT'].mean()
bldd=sample.groupby('TAXCLASS')['BLDDEPTH'].mean()

In [27]:
for i in ltf.index:
    sample.loc[(sample['LTFRONT'].isnull())&(sample['TAXCLASS']==i),'LTFRONT']=ltf[i]
    sample.loc[(sample['LTDEPTH'].isnull())&(sample['TAXCLASS']==i),'LTDEPTH']=ltd[i]
    sample.loc[(sample['BLDFRONT'].isnull())&(sample['TAXCLASS']==i),'BLDFRONT']=bldf[i]
    sample.loc[(sample['BLDDEPTH'].isnull())&(sample['TAXCLASS']==i),'BLDDEPTH']=bldd[i]
    

In [28]:
original_data=sample.copy()

### Variable creation

In [29]:
sample['ZIP']=sample['ZIP'].astype(str)
sample['zip3']=sample['ZIP'].str[:3]

In [30]:
sample['ltsize']=sample['LTFRONT']*sample['LTDEPTH']
sample['bldsize']=sample['BLDFRONT']*sample['BLDDEPTH']
sample['bldvol']=sample['bldsize']*sample['STORIES']

In [31]:
sample['r1']=sample['FULLVAL']/sample['ltsize']
sample['r2']=sample['FULLVAL']/sample['bldsize']
sample['r3']=sample['FULLVAL']/sample['bldvol']
sample['r4']=sample['AVLAND']/sample['ltsize']
sample['r5']=sample['AVLAND']/sample['bldsize']
sample['r6']=sample['AVLAND']/sample['bldvol']
sample['r7']=sample['AVTOT']/sample['ltsize']
sample['r8']=sample['AVTOT']/sample['bldsize']
sample['r9']=sample['AVTOT']/sample['bldvol']

In [32]:
for col in sample.columns[36:]:
    sample[col+'inv']=1/(sample[col])

In [33]:
#sample.head().transpose()

In [34]:
var=['r1','r2','r3','r4','r5','r6','r7','r8','r9','r1inv', 'r2inv',
       'r3inv', 'r4inv', 'r5inv', 'r6inv', 'r7inv', 'r8inv', 'r9inv']
zipm=sample.groupby('ZIP')[var].mean()
zip3m=sample.groupby('zip3')[var].mean()
taxm=sample.groupby('TAXCLASS')[var].mean()
borom=sample.groupby('BORO')[var].mean()

In [35]:
sample=sample.join(zipm,on='ZIP',rsuffix='_zip5')
sample=sample.join(zip3m,on='zip3',rsuffix='_zip3')
sample=sample.join(taxm,on='TAXCLASS',rsuffix='_taxclass')
sample=sample.join(borom,on='BORO',rsuffix='_boro')

rsuffix=['_zip5','_zip3','_taxclass','_boro']
for v in var:
    for r in rsuffix:
        sample[str(v)+r]=sample[v]/sample[str(v)+r]

In [36]:
# sample.head().transpose()

In [37]:
sample.columns

Index(['RECORD', 'BBLE', 'BORO', 'BLOCK', 'LOT', 'EASEMENT', 'OWNER', 'BLDGCL',
       'TAXCLASS', 'LTFRONT',
       ...
       'r9_boro', 'r1inv_boro', 'r2inv_boro', 'r3inv_boro', 'r4inv_boro',
       'r5inv_boro', 'r6inv_boro', 'r7inv_boro', 'r8inv_boro', 'r9inv_boro'],
      dtype='object', length=126)

In [38]:
sample2=sample.copy()


In [39]:
# sample.drop(('RECORD', 'BBLE', 'BORO', 'BLOCK', 'LOT', 'EASEMENT', 'OWNER', 'BLDGCL',
#        'TAXCLASS', 'LTFRONT', 'LTDEPTH', 'EXT', 'STORIES', 'FULLVAL', 'AVLAND',
#        'AVTOT', 'EXLAND', 'EXTOT', 'EXCD1', 'STADDR', 'ZIP', 'EXMPTCL',
#        'BLDFRONT', 'BLDDEPTH', 'AVLAND2', 'AVTOT2', 'EXLAND2', 'EXTOT2',
#        'EXCD2', 'PERIOD', 'YEAR', 'VALTYPE', 'zip3', 'ltsize', 'bldsize',
#        'bldvol'),axis=1)

In [40]:
x=['RECORD', 'BBLE', 'BORO', 'BLOCK', 'LOT', 'EASEMENT', 'OWNER', 'BLDGCL',
       'TAXCLASS', 'LTFRONT', 'LTDEPTH', 'EXT', 'STORIES', 'FULLVAL', 'AVLAND',
       'AVTOT', 'EXLAND', 'EXTOT', 'EXCD1', 'STADDR', 'ZIP', 'EXMPTCL',
       'BLDFRONT', 'BLDDEPTH', 'AVLAND2', 'AVTOT2', 'EXLAND2', 'EXTOT2',
       'EXCD2', 'PERIOD', 'YEAR', 'VALTYPE', 'zip3', 'ltsize', 'bldsize',
       'bldvol']
for i in x:
    sample=sample.drop(i,axis=1)

In [41]:
sample

,r1,r2,r3,r4,r5,r6,r7,r8,r9,r1inv,...,r9_boro,r1inv_boro,r2inv_boro,r3inv_boro,r4inv_boro,r5inv_boro,r6inv_boro,r7inv_boro,r8inv_boro,r9inv_boro
0,40.917782,3994.412456,723.985188,8.079350,788.709805,142.953243,18.413002,1797.485605,325.793334,0.024439,...,14.957255,0.102422,0.001656,0.000445,0.038071,0.000569,0.000147,0.106805,0.001731,0.000459
1,57678.882190,36173.697854,6556.463990,4258.951518,2671.030012,484.122805,25955.496986,16278.164034,2950.408795,0.000017,...,135.454020,0.000073,0.000183,0.000049,0.000072,0.000168,0.000043,0.000076,0.000191,0.000051
2,385.182796,736.308714,14.726174,62.110738,118.729804,2.374596,173.332268,331.338940,6.626779,0.002596,...,0.304237,0.010880,0.008984,0.021861,0.004952,0.003782,0.008830,0.011346,0.009388,0.022555
3,556.662267,1064.105879,21.282118,89.761892,171.587267,3.431745,250.498069,478.847739,9.576955,0.001796,...,0.439680,0.007529,0.006217,0.015127,0.003427,0.002617,0.006110,0.007851,0.006496,0.015607
4,1035.027624,1978.540749,39.570815,94.470682,180.588507,3.611770,465.762431,890.343337,17.806867,0.000966,...,0.817518,0.004049,0.003343,0.008136,0.003256,0.002487,0.005805,0.004222,0.003494,0.008394
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1046821,60.833333,561.538462,224.615385,2.018889,18.635897,7.454359,3.239167,29.900000,11.960000,0.016438,...,0.685258,0.084219,0.009161,0.004135,0.198868,0.056347,0.031206,0.152256,0.040521,0.021753
1046822,38.483305,653.846154,261.538462,0.658107,11.181490,4.472596,1.056098,17.943510,7.177404,0.025985,...,0.411235,0.133130,0.007867,0.003551,0.610070,0.093912,0.052010,0.466985,0.067521,0.036248
1046823,33.174373,625.000000,312.500000,1.022185,19.257812,9.628906,1.587878,29.915365,14.957682,0.030144,...,0.857013,0.154435,0.008231,0.002972,0.392778,0.054527,0.024159,0.310592,0.040500,0.017394
1046824,76.781674,808.035714,404.017857,1.751697,18.434524,9.217262,3.257494,34.281250,17.140625,0.013024,...,0.982087,0.066726,0.006366,0.002299,0.229201,0.056963,0.025238,0.151399,0.035342,0.015178


In [42]:
sample.shape

(1046826, 90)

In [43]:
output=sample.describe()

### Homework 3 begins

#### Z score

In [44]:
sample_zs=(sample-sample.mean())/sample.std()
sample_zs_save=sample_zs.copy()
sample_zs.describe()

,r1,r2,r3,r4,r5,r6,r7,r8,r9,r1inv,...,r9_boro,r1inv_boro,r2inv_boro,r3inv_boro,r4inv_boro,r5inv_boro,r6inv_boro,r7inv_boro,r8inv_boro,r9inv_boro
count,1.046826e+06,1.046826e+06,1.046826e+06,1.046826e+06,1.046826e+06,1.046826e+06,1.046826e+06,1.046826e+06,1.046826e+06,1.046826e+06,...,1.046826e+06,1.046826e+06,1.046826e+06,1.046826e+06,1.046826e+06,1.046826e+06,1.046826e+06,1.046826e+06,1.046826e+06,1.046826e+06
mean,-1.836969e-14,-3.051849e-14,-3.563055e-14,7.322379e-16,2.313336e-15,1.989244e-15,-4.578153e-14,-1.491621e-15,-3.818909e-15,-4.955384e-15,...,-3.116924e-15,1.627437e-15,-4.801006e-16,1.725544e-15,7.515213e-16,-1.388540e-15,-1.860504e-16,-9.458200e-16,4.386177e-15,1.296240e-15
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-4.010199e-01,-5.220872e-01,-4.893158e-01,-1.495467e-01,-6.028049e-02,-5.630825e-02,-1.316171e-01,-5.413210e-02,-5.832592e-02,-1.948726e-02,...,-6.358523e-02,-1.746644e-02,-2.238987e-02,-2.092592e-02,-1.964540e-02,-5.104402e-02,-3.400143e-02,-1.596278e-02,-4.039296e-02,-4.628516e-02
25%,-2.578217e-01,-3.159291e-01,-3.239267e-01,-1.159055e-01,-4.388961e-02,-4.500406e-02,-1.057139e-01,-3.539445e-02,-3.664775e-02,-1.878025e-02,...,-3.874582e-02,-1.662974e-02,-2.174872e-02,-2.078692e-02,-1.801363e-02,-4.811786e-02,-3.354228e-02,-1.362558e-02,-3.706528e-02,-4.479725e-02
50%,-1.139995e-01,-3.544288e-02,-1.006284e-02,-8.385372e-02,-2.070184e-02,-1.623124e-02,-8.953595e-02,-2.519297e-02,-1.955740e-02,-1.836174e-02,...,-2.079726e-02,-1.619112e-02,-2.145329e-02,-2.070205e-02,-1.687643e-02,-4.700363e-02,-3.318353e-02,-1.120199e-02,-3.527223e-02,-4.388735e-02
75%,5.560400e-02,1.434707e-01,1.847335e-01,-4.629056e-02,-6.684979e-03,1.259701e-05,-6.526729e-02,-1.551410e-02,-4.581517e-03,-1.723007e-02,...,-3.857554e-03,-1.502318e-02,-2.042848e-02,-2.039314e-02,-1.455371e-02,-4.340098e-02,-3.214041e-02,-8.155987e-03,-3.209285e-02,-4.227557e-02
max,2.599196e+02,3.017347e+02,4.555842e+02,3.178144e+02,8.840365e+02,6.254489e+02,3.172796e+02,9.346833e+02,8.881159e+02,4.409551e+02,...,8.786441e+02,6.237607e+02,7.342993e+02,7.765578e+02,9.464886e+02,1.749722e+02,1.052204e+02,9.446987e+02,5.572700e+02,4.767697e+02


### Principle Component Analysis 

In [45]:
from sklearn.decomposition import PCA

# pca=PCA(n_components=0.99,svd_solver='full')  #if 0 < n_components < 1 and svd_solver == 'full', select the number of components such that the amount of variance that needs to be explained is greater than the percentage specified by n_components.
# pca.fit(sample_zs)
# pca.explained_variance_ratio_         #Percentage of variance explained by each of the selected components.

In [46]:
# t=pca.explained_variance_ratio_

In [47]:
# plt.plot(n.cumsum(t))
# plt.xlabel('Number of components')
# plt.ylabel('Cumulative Variance explained')
# plt.axvline(x=6,linestyle='--')

### Performing PCA again with 7 components

In [48]:
pca=PCA(n_components=7,svd_solver='full')
#sample_zs=sample_zs_save.copy()

In [49]:
z=pca.fit_transform(sample_zs)

In [50]:
z.shape

(1046826, 7)

In [51]:
pca.n_components_

7

In [52]:
sample_pca=p.DataFrame(z,columns=['PC'+str(i) for i in range(1,pca.n_components_+1)])

In [53]:
sample_pca.head()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7
0,7.812986,0.782459,-1.692937,0.098349,0.809687,-4.945516,2.719589
1,118.084854,9.786138,199.378118,0.399513,6.409308,-47.300143,27.856300
2,0.236833,-0.261303,0.707700,0.005826,0.003594,0.316049,-0.160373
3,0.834013,-0.204194,1.435032,0.008552,0.035698,0.083755,-0.026838
4,2.085611,-0.089649,2.951878,0.020970,0.159003,-0.774737,0.453820


In [54]:
sample_pca.describe()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7
count,1.046826e+06,1.046826e+06,1.046826e+06,1.046826e+06,1.046826e+06,1.046826e+06,1.046826e+06
mean,9.340193e-16,9.059290e-17,-2.352618e-16,2.170314e-16,-1.384463e-15,3.536920e-16,-9.440078e-17
std,4.448435e+00,4.251885e+00,3.374752e+00,2.587648e+00,2.308366e+00,2.277909e+00,2.242323e+00
min,-2.293067e+02,-3.645505e-01,-7.664554e+02,-7.165028e+02,-7.013362e+02,-5.260210e+02,-7.315593e+02
25%,-4.607729e-01,-2.158022e-01,-5.202718e-01,-6.543507e-03,-4.904047e-02,-3.033607e-01,-2.874438e-01
50%,-7.852356e-02,-1.900084e-01,-2.034711e-01,6.802326e-03,2.332644e-02,-4.028889e-02,-6.883125e-02
75%,1.745691e-01,-1.520397e-01,1.047161e-01,2.419782e-02,7.298279e-02,6.243616e-01,8.824867e-02
max,3.036422e+03,1.683254e+03,1.093464e+03,2.126744e+03,1.020287e+03,1.181004e+03,7.825028e+02


#### Apply z scaling on new 7 features

In [55]:
sample_pca_zs=(sample_pca-sample_pca.mean())/sample_pca.std()
sample_pca_zs.describe()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7
count,1.046826e+06,1.046826e+06,1.046826e+06,1.046826e+06,1.046826e+06,1.046826e+06,1.046826e+06
mean,-8.487581e-16,6.580823e-17,-1.987780e-15,1.506013e-17,7.957568e-16,-7.852879e-16,-2.383392e-16
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-5.154774e+01,-8.573855e-02,-2.271146e+02,-2.768935e+02,-3.038236e+02,-2.309228e+02,-3.262506e+02
25%,-1.035809e-01,-5.075447e-02,-1.541659e-01,-2.528747e-03,-2.124467e-02,-1.331751e-01,-1.281902e-01
50%,-1.765195e-02,-4.468804e-02,-6.029215e-02,2.628768e-03,1.010517e-02,-1.768679e-02,-3.069640e-02
75%,3.924282e-02,-3.575819e-02,3.102926e-02,9.351279e-03,3.161665e-02,2.740942e-01,3.935591e-02
max,6.825822e+02,3.958843e+02,3.240133e+02,8.218831e+02,4.419954e+02,5.184597e+02,3.489697e+02


In [56]:
sample_pca_zs.head(20)

,PC1,PC2,PC3,PC4,PC5,PC6,PC7
0,1.756345,0.184026,-0.501648,0.038007,0.350762,-2.171078,1.212844
1,26.545259,2.301600,59.079339,0.154392,2.776556,-20.764725,12.422965
2,0.053240,-0.061456,0.209704,0.002252,0.001557,0.138745,-0.071521
3,0.187485,-0.048024,0.425226,0.003305,0.015465,0.036769,-0.011969
4,0.468842,-0.021084,0.874695,0.008104,0.068881,-0.340109,0.202388
5,-0.171345,-0.080997,-0.150135,-0.000186,-0.020862,0.310035,-0.169250
6,-0.071180,-0.073393,0.010082,0.001157,-0.011185,0.233379,-0.126383
7,-0.080862,-0.074268,-0.005439,0.001060,-0.012161,0.240755,-0.130617
8,0.342613,-0.032289,0.674327,0.004473,0.031599,-0.081022,0.056986
9,0.326877,-0.033891,0.649056,0.004356,0.029960,-0.069074,0.049988


### Model 1 

In [57]:
p1=2
in_p1=1/p1
score1=(((sample_pca_zs).abs()**p1).sum(axis=1))**in_p1

In [58]:
score1

0           3.111159
1          69.235492
2           0.273789
3           0.469062
4           1.070884
             ...    
1046821     0.211109
1046822     0.291675
1046823     0.286916
1046824     0.352972
1046825     0.203658
Length: 1046826, dtype: float64

In [59]:
score1.head(10)

0     3.111159
1    69.235492
2     0.273789
3     0.469062
4     1.070884
5     0.428559
6     0.284815
7     0.295393
8     0.764182
9     0.733115
dtype: float64

### Model 2

In [60]:
from sklearn.neural_network import MLPRegressor
model=MLPRegressor(hidden_layer_sizes=(10),activation='relu',verbose=True,max_iter=15,random_state=1)
model.fit(sample_pca_zs,sample_pca_zs)

Iteration 1, loss = 0.31203909
Iteration 2, loss = 0.15880255
Iteration 3, loss = 0.10783790
Iteration 4, loss = 0.08697483
Iteration 5, loss = 0.05928352
Iteration 6, loss = 0.03834116
Iteration 7, loss = 0.02695220
Iteration 8, loss = 0.01765546
Iteration 9, loss = 0.01269320
Iteration 10, loss = 0.00949307
Iteration 11, loss = 0.00595564
Iteration 12, loss = 0.00330296
Iteration 13, loss = 0.00309626
Iteration 14, loss = 0.00168802
Iteration 15, loss = 0.00106744


/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPRegressor(hidden_layer_sizes=10, max_iter=15, random_state=1, verbose=True)

In [61]:
NNmodel = MLPRegressor(hidden_layer_sizes = (10),activation = 'relu',verbose = True, max_iter = 15, random_state =1)
NNmodel.fit(sample_pca_zs,sample_pca_zs)

Iteration 1, loss = 0.31203909
Iteration 2, loss = 0.15880255
Iteration 3, loss = 0.10783790
Iteration 4, loss = 0.08697483
Iteration 5, loss = 0.05928352
Iteration 6, loss = 0.03834116
Iteration 7, loss = 0.02695220
Iteration 8, loss = 0.01765546
Iteration 9, loss = 0.01269320
Iteration 10, loss = 0.00949307
Iteration 11, loss = 0.00595564
Iteration 12, loss = 0.00330296
Iteration 13, loss = 0.00309626
Iteration 14, loss = 0.00168802
Iteration 15, loss = 0.00106744


/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPRegressor(hidden_layer_sizes=10, max_iter=15, random_state=1, verbose=True)

In [62]:
pca_out=model.predict(sample_pca_zs)
error=pca_out-sample_pca_zs
p2=2
in_p2=1/p2
score2=(error.abs()**p2).sum(axis = 1)**in_p2

In [63]:
score2

0          0.032713
1          1.062363
2          0.023396
3          0.025584
4          0.030607
             ...   
1046821    0.009484
1046822    0.042135
1046823    0.026401
1046824    0.012625
1046825    0.005897
Length: 1046826, dtype: float64

In [64]:
pca_out = NNmodel.predict(sample_pca_zs)
error = pca_out - sample_pca_zs
p2 = 2
oop2 = 1/p2
k = (error.abs()**p2).sum(axis = 1)**oop2
k.head()

0    0.032713
1    1.062363
2    0.023396
3    0.025584
4    0.030607
dtype: float64

In [65]:
scores=p.DataFrame(score1)
scores.columns=(['score1'])
scores['score2']=score2


In [66]:
scores['score1rank']=scores['score1'].rank()
scores['score2rank']=scores['score2'].rank()

In [67]:
scores['final']=(0.5*scores['score1rank']+0.5*scores['score2rank'])

In [68]:
scores

,score1,score2,score1rank,score2rank,final
0,3.111159,0.032713,1033738.0,924622.0,979180.0
1,69.235492,1.062363,1046696.0,1046048.0,1046372.0
2,0.273789,0.023396,573498.0,762788.0,668143.0
3,0.469062,0.025584,752401.0,809187.0,780794.0
4,1.070884,0.030607,1004165.0,900752.0,952458.5
...,...,...,...,...,...
1046821,0.211109,0.009484,463522.0,372292.0,417907.0
1046822,0.291675,0.042135,594743.0,973934.0,784338.5
1046823,0.286916,0.026401,589387.0,826518.0,707952.5
1046824,0.352972,0.012625,653958.0,493754.0,573856.0


In [69]:
top_records = scores.sort_values(['final'], ascending = False).head(30).index

top_records

Int64Index([ 975628,  898846,  675033,  701673,  633064, 1031078,  759739,
             936080,  642176,    7010,  974507,    7016,    7015,    7014,
               7013,    7012,    7011,    7018,    7017,    7009,    7019,
             444326,  914593,  232192,  113171, 1036415, 1026195,  827753,
             231106,  979841],
           dtype='int64')

In [70]:
original_data[['score1','score2']]=scores[['score1','score2']]

In [71]:
original_data=original_data.drop(['score1'],axis=1)
original_data=original_data.drop(['score2'],axis=1)

In [72]:
original_data[['score1 rank','score2 rank','final']]=scores[['score1rank','score2rank','final']]
ans[['score1 rank','score2 rank','final']]=scores[['score1rank','score2rank','final']]

In [73]:
ans.reindex(top_records)

,RECORD,BBLE,BORO,BLOCK,LOT,EASEMENT,OWNER,BLDGCL,TAXCLASS,LTFRONT,...,AVTOT2,EXLAND2,EXTOT2,EXCD2,PERIOD,YEAR,VALTYPE,score1 rank,score2 rank,final
975628,996722,5029220125,5,2922,125,NaN,IMPERIAL COURT HOMEOW,V0,1B,300,...,NaN,NaN,NaN,NaN,FINAL,2010/11,AC-TR,1046825.0,1046826.0,1046825.5
898846,917942,4142600001,4,14260,1,NaN,"LOGAN PROPERTY, INC.",T1,4,4910,...,4.501180e+09,1.644454e+09,4.501180e+09,NaN,FINAL,2010/11,AC-TR,1046826.0,1046825.0,1046825.5
675033,691986,4039160200,4,3916,200,NaN,TITAN SOUNDVIEW POINT,Z7,4,0,...,3.000000e+00,NaN,NaN,NaN,FINAL,2010/11,AC-TR,1046824.0,1046823.0,1046823.5
701673,718883,4051251107,4,5125,1107,NaN,GARDEN VIEW LTD,R4,2,0,...,NaN,NaN,NaN,NaN,FINAL,2010/11,AC-TR,1046822.0,1046821.0,1046821.5
633064,649717,4025270002,4,2527,2,NaN,57-43 LLC,V1,4,51,...,NaN,NaN,NaN,NaN,FINAL,2010/11,AC-TR,1046819.0,1046824.0,1046821.5
1031078,1053859,5064310100,5,6431,100,NaN,NaN,Z7,4,1000,...,1.390000e+02,NaN,NaN,NaN,FINAL,2010/11,AC-TR,1046810.0,1046822.0,1046816.0
759739,777200,4080990145E,4,8099,145,E,NY STATE PUBLIC WORKS,Z7,4,110,...,NaN,NaN,NaN,NaN,FINAL,2010/11,AC-TR,1046817.0,1046808.0,1046812.5
936080,956520,5006590012,5,659,12,NaN,TROMPETA RIZALINA,A1,1,25,...,NaN,NaN,NaN,NaN,FINAL,2010/11,AC-TR,1046823.0,1046798.0,1046810.5
642176,658933,4029060054,4,2906,54,NaN,WAN CHIU CHEUNG,C0,1,25,...,NaN,NaN,NaN,NaN,FINAL,2010/11,AC-TR,1046820.0,1046799.0,1046809.5
7010,7034,1000531146,1,53,1146,NaN,"HSIA, JONATHAN",R4,2,0,...,NaN,NaN,NaN,NaN,FINAL,2010/11,AC-TR,1046803.0,1046815.0,1046809.0


In [74]:
#original_data.reindex()

In [75]:
original_data.sort_values('final',ascending=False).head(30)

,RECORD,BBLE,BORO,BLOCK,LOT,EASEMENT,OWNER,BLDGCL,TAXCLASS,LTFRONT,...,AVTOT2,EXLAND2,EXTOT2,EXCD2,PERIOD,YEAR,VALTYPE,score1 rank,score2 rank,final
975628,996722,5029220125,5,2922,125,NaN,IMPERIAL COURT HOMEOW,V0,1B,300.000000,...,NaN,NaN,NaN,NaN,FINAL,2010/11,AC-TR,1046825.0,1046826.0,1046825.5
898846,917942,4142600001,4,14260,1,NaN,"LOGAN PROPERTY, INC.",T1,4,4910.000000,...,4.501180e+09,1.644454e+09,4.501180e+09,NaN,FINAL,2010/11,AC-TR,1046826.0,1046825.0,1046825.5
675033,691986,4039160200,4,3916,200,NaN,TITAN SOUNDVIEW POINT,Z7,4,82.296384,...,3.000000e+00,NaN,NaN,NaN,FINAL,2010/11,AC-TR,1046824.0,1046823.0,1046823.5
701673,718883,4051251107,4,5125,1107,NaN,GARDEN VIEW LTD,R4,2,111.579150,...,NaN,NaN,NaN,NaN,FINAL,2010/11,AC-TR,1046822.0,1046821.0,1046821.5
633064,649717,4025270002,4,2527,2,NaN,57-43 LLC,V1,4,51.000000,...,NaN,NaN,NaN,NaN,FINAL,2010/11,AC-TR,1046819.0,1046824.0,1046821.5
1031078,1053859,5064310100,5,6431,100,NaN,NaN,Z7,4,1000.000000,...,1.390000e+02,NaN,NaN,NaN,FINAL,2010/11,AC-TR,1046810.0,1046822.0,1046816.0
759739,777200,4080990145E,4,8099,145,E,NY STATE PUBLIC WORKS,Z7,4,110.000000,...,NaN,NaN,NaN,NaN,FINAL,2010/11,AC-TR,1046817.0,1046808.0,1046812.5
936080,956520,5006590012,5,659,12,NaN,TROMPETA RIZALINA,A1,1,25.000000,...,NaN,NaN,NaN,NaN,FINAL,2010/11,AC-TR,1046823.0,1046798.0,1046810.5
642176,658933,4029060054,4,2906,54,NaN,WAN CHIU CHEUNG,C0,1,25.000000,...,NaN,NaN,NaN,NaN,FINAL,2010/11,AC-TR,1046820.0,1046799.0,1046809.5
7010,7034,1000531146,1,53,1146,NaN,"HSIA, JONATHAN",R4,2,111.579150,...,NaN,NaN,NaN,NaN,FINAL,2010/11,AC-TR,1046803.0,1046815.0,1046809.0


In [76]:
#original_data.reindex(t)

In [ ]:
ans.set_index('RECORD').to_excel("Final_df_Sudhanshu_Rai.xlsx")